In [ ]:
from pwv_kpno import __version__
if not int(__version__.split('.')[1]) >= 12:
    raise ImportError('Requires at least pwv_kpno version 0.12.0')


## Creating a new config file

In order to model the atmosphere at a given location, `pwv_kpno` requires a config file for that location. We can create a custom config file using the `ConfigBuilder` class as follows:

In [ ]:
import numpy as np
from pwv_kpno import ConfigBuilder


# An subset of the Kitt Peak model using Angstroms and cm^2
wavelengths = np.array([3000, 3001, 3002]) 
modtran_cross_sections = np.array([5.719913264564206E-9,
                                   5.725763008721962E-9,
                                   5.7315124715513006E-9])

new_config = ConfigBuilder()
new_config.loc_name = 'example_loc'
new_config.primary_rec = 'KITT' # The primary receiver
new_config.sup_rec = ['AZAM']  # A list of secondary receivers
new_config.wavelengths = wavelengths
new_config.cross_sections = modtran_cross_sections

# Create output file example_loc.ecsv
# By default existing files are not overwritten
new_config.save_to_dir('.', overwrite=True)


An alternative but functionally equivilent method would be to instantiate the class with the desired information:

In [ ]:
new_config = ConfigBuilder(
    loc_name = 'example_loc',
    primary_rec = 'KITT',
    sup_rec = ['AZAM'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections
)

new_config.save_to_dir('.', overwrite=True)

If desired, useres can specify custom data cuts on SuomiNet data used by the package. Data cuts can be specified using a 2d dictionary of boundary values. The first key specifies which receiver the data cuts apply to. The second key specifies what values to cut. Following SuomiNet's naming convention, optional values that can be cut include `'PWV', 'PWVerr', 'ZenithDelay', 'SrfcPress', 'SrfcTemp',` and `'SrfcRH'`. For example, if we only wanted to use pressure measurements between 880 and 925 mbar for the AZAM receiver:

In [ ]:
# Here we define what data NOT to use
data_cuts = {'AZAM': {'SrfcPress': [[0, 880], [925, 10000]]}}     

new_config = ConfigBuilder(
    loc_name = 'example_loc',
    primary_rec = 'KITT',
    sup_rec = ['AZAM'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections,
    data_cuts = data_cuts
)

new_config.save_to_dir('.', overwrite=True)

## Errors and Warnings

If the user forgets to provide any required information, `save_to_dir` will raise an error.

In [ ]:
new_config = ConfigBuilder(
    primary_rec = 'kitt',
    sup_rec = ['Sa48'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections
)

new_config.save_to_dir('.', overwrite=True)

Any issues with formatting the data will raise warnings. This helps ensure that the user understands how the package works. These warnings are also raised when the `save_to_dir` method is called.

In [ ]:
new_config = ConfigBuilder(
    loc_name = 'EXAMPLE_LOC',
    primary_rec = 'kitt',
    sup_rec = ['Sa48'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections
)

new_config.save_to_dir('.', overwrite=True)

# Choosing a location into pwv_kpno

When imported, the `pwv_kpno` package has no knowledge of what location you wish to model

In [ ]:
from pwv_kpno import pwv_atm

pwv_atm.available_data()

We can configure `pwv_kpno` to model a specific location using the `settings` object

In [ ]:
from pwv_kpno import settings

settings.set_location('kitt_peak')
pwv_atm.available_data()

The `settings` object also provides a handful of useful information about the current location

In [ ]:
print('Location being modeled:', settings.loc_name)
print('List of available locations: ', settings.available_loc)
print('List of receiver ids for the current location: ', settings.receivers)
print('The id for primary receiver: ', settings.primary_rec)
print('The ids for supplementary receivers: ', settings.supplement_rec)

In order to model a custom location we first need to import the appropriate config file. Note that this step only has to be done once. Afterwards we can switch betwean locations normally using the `set_location` method.

In [ ]:
#settings.import_location('./example_loc.ecsv')
settings.set_location('example_loc')
pwv_atm.available_data()

By default there is no local SuomiNet data for this location. We need to download this first

In [ ]:
pwv_atm.update_models(2010)